This document will contain the template and codes for creating master data dictionary , carrying out basic and advanced
text analytics to prepare training set data be be feeeded into an machine learning supervised model to classify
product descriptions into multi level multi class classification/categories. 

Creating the Master Data Dictionary of Product Master Classes (Nouns)
=====================================


In [1]:
# Importing all necessary libraries
import time
start = time.time()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [2]:
# Import the dataset contaning the texts showing mapping between all possible  Noun,Modifier and Characteristic
raw_data_catelogue = pd.read_csv('Master_Dictionary.csv')
raw_data_catelogue.drop_duplicates(subset=None, inplace=True)
raw_data_catelogue = raw_data_catelogue.fillna('NA')

In [4]:

raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [5]:
import warnings
warnings.filterwarnings('ignore')

defDict = defaultdict(LabelEncoder)
data_dictionary_encoded = raw_data_catelogue.apply(lambda x: defDict[x.name].fit_transform(x))
orig_raw_data_catelogue = data_dictionary_encoded.apply(lambda x: defDict[x.name].inverse_transform(x))

In [6]:
orig_raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [7]:
data_dictionary_encoded.head()

,Noun,Modifier,Characteristic,TheValue
0,0,2,616,15012
1,0,54,88,13252
2,0,54,88,16157
3,0,54,88,16314
4,0,54,88,16322


In [8]:
data_dictionary_mapping = pd.concat([orig_raw_data_catelogue,data_dictionary_encoded], axis=1)
headers=['Noun','Modifier','Characteristic','TheValue','Noun_en','Modifier_en','Characteristic_en','TheValue_en']
data_dictionary_mapping.columns=headers

In [9]:
data_dictionary_mapping.head()

,Noun,Modifier,Characteristic,TheValue,Noun_en,Modifier_en,Characteristic_en,TheValue_en
0,ABRASIVE,AAAA,NA,NA,0,2,616,15012
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND,0,54,88,13252
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING,0,54,88,16157
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND,0,54,88,16314
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED",0,54,88,16322


In [10]:
#Saving the dictionary to a file for reference
data_dictionary_mapping.to_csv('data_dictionary_mapping.csv', sep=',')

Basic feature extraction using text data
========================================

In [6]:
#Import the semi clean dataset.Exporting only the short description
dtype_dic = {'DESCRIPTION': str,'Noun':str,'Modifier':str}
txt_corpus = pd.read_csv('Training_Data6.csv', dtype=dtype_dic,encoding = "latin")
txt_corpus.drop_duplicates(subset=None, inplace=True)
txt_corpus = txt_corpus.replace(r'\r',' ', regex=True)
txt_corpus.reset_index(drop=True, inplace=True)

In [7]:
txt_corpus.shape

(550861, 4)

In [13]:
txt_corpus['Modifier'].value_counts()

NO MODIFIER        263389
ELECTRICAL          13982
PNEUMATIC            9812
BALL                 9520
PIPE                 9033
AC                   7666
ROLLER               7085
OIL                  6773
ELECTRIC             5840
SOLENOID             5657
BEARING              5337
PRESSURE             5223
VEE                  5182
CONVEYOR             4315
PROXIMITY            4288
MACHINE              4083
ELEMENT              3647
DRIVE                3189
CABLE                3173
CAP                  2977
PHOTOELECTRIC        2881
COMPRESSION          2782
TUBE TO PIPE         2729
ASSEMBLY             2712
VALVE                2685
TOOTHED              2659
BRICK                2623
PRINTED CIRCUIT      2531
SEALING              2457
TUBING               2245
                    ...  
OCTAGON                 1
BUMPER                  1
TONGUE                  1
SMOKE DETECTOR          1
BURRING                 1
CAN                     1
NOISE                   1
ACOUSTICAL  

In [13]:
print(txt_corpus.shape)
txt_corpus = txt_corpus.sample(frac=1)
txt_corpus = txt_corpus[txt_corpus['Modifier'] != 'NO MODIFIER']
txt_corpus = txt_corpus[txt_corpus['Modifier'] != 'NA']
txt_corpus = txt_corpus[txt_corpus['Noun'] != 'NA']
print(txt_corpus.shape)

(21077, 3)
(21077, 3)


In [14]:
txt_corpus.reset_index(drop=True, inplace=True)
txt_corpus['DESCRIPTION'] = txt_corpus['DESCRIPTION'].replace(r'_____PO TEXT_____',' ', regex=True)
txt_corpus['DESCRIPTION'] = txt_corpus['DESCRIPTION'].replace(r'_____CLASS_____',' ', regex=True)
txt_corpus.head()

,Noun,Modifier,DESCRIPTION
0,GASKET,CONTROL VALVE,VALVES\n\n\nSPARES FOR FLOWSERVE CONTROL VALVE...
1,OIL,INDUSTRIAL,"OIL,HYDRUALIC,RANDO HD 46,INFLAMMABLE\n\n \nPA..."
2,GEAR,HELICAL,"UNIT,HELICAL GEAR,SK1SI75VH.IEC.80\n\n \nMOTOR..."
3,SPLICE,CONDUCTOR,"SLEEVE,CABLE,TYPE-SB8, DE-3BEXZSB08000\n\n \nF..."
4,VALVE,BUTTERFLY,"VALVE,BUTTERFLY,PNEUMATIC OPERATED,8IN\n\n \nW..."


In [15]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
txt_corpus_clean_desc=[]
txt_corpus_clean_desc = pd.DataFrame(columns=['clean_data'])
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def punctutaion_removal(text):
    s = re.sub(r'[^\w\s]',' ',text)
    return s

txt_corpus['clean_data'] = txt_corpus.DESCRIPTION.apply(punctutaion_removal)

stop = stopwords.words('english')
txt_corpus['clean_data'] = txt_corpus['clean_data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

def lemmatize_text(text):
    new_text = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return(' '.join(new_text))

txt_corpus['clean_data'] = txt_corpus['clean_data'].apply(lemmatize_text)
print(txt_corpus['clean_data'].head())

0    VALVES SPARES FOR FLOWSERVE CONTROL VALVE SIZE...
1    OIL HYDRUALIC RANDO HD 46 INFLAMMABLE PACKING ...
2    UNIT HELICAL GEAR SK1SI75VH IEC 80 MOTORIZED R...
3    SLEEVE CABLE TYPE SB8 DE 3BEXZSB08000 FOR MILL...
4    VALVE BUTTERFLY PNEUMATIC OPERATED 8IN WITHOUT...
Name: clean_data, dtype: object


In [16]:
# df_noun_modifier = pd.DataFrame(columns=['noun','modifier'])
master_txt_corpus = pd.DataFrame()
master_txt_corpus = txt_corpus
master_txt_corpus.head()

,Noun,Modifier,DESCRIPTION,clean_data
0,GASKET,CONTROL VALVE,VALVES\n\n\nSPARES FOR FLOWSERVE CONTROL VALVE...,VALVES SPARES FOR FLOWSERVE CONTROL VALVE SIZE...
1,OIL,INDUSTRIAL,"OIL,HYDRUALIC,RANDO HD 46,INFLAMMABLE\n\n \nPA...",OIL HYDRUALIC RANDO HD 46 INFLAMMABLE PACKING ...
2,GEAR,HELICAL,"UNIT,HELICAL GEAR,SK1SI75VH.IEC.80\n\n \nMOTOR...",UNIT HELICAL GEAR SK1SI75VH IEC 80 MOTORIZED R...
3,SPLICE,CONDUCTOR,"SLEEVE,CABLE,TYPE-SB8, DE-3BEXZSB08000\n\n \nF...",SLEEVE CABLE TYPE SB8 DE 3BEXZSB08000 FOR MILL...
4,VALVE,BUTTERFLY,"VALVE,BUTTERFLY,PNEUMATIC OPERATED,8IN\n\n \nW...",VALVE BUTTERFLY PNEUMATIC OPERATED 8IN WITHOUT...


Bag of words Model
==================

In [17]:
#words = " ".join(master_txt_corpus.clean_data).split()
#words
all_words = {x for x in ' '.join(master_txt_corpus.clean_data.str.upper().tolist()).split() if x.isalpha()} 
all_words_bag_freq = nltk.FreqDist(all_words)
total_words=len(set(all_words))
# frequencies_words = nltk.FreqDist(words).most_common()
# print(frequencies_words)

In [18]:
print(all_words_bag_freq["GASKET"])

1


Funtion to derive features if given a text.Features would correspond to the words picked up from bag of words.
=================================================================================================================
We assume a 40 significant word feature extraction from a given text.
=================================================================================================================

In [19]:
word_features = set(map(str.upper,list(all_words_bag_freq.keys())[:total_words]))
#set(str.upper(list(all_words_bag_freq.keys())[:40]))
print(len(word_features))

def find_features(prod_desc):
    words = nltk.word_tokenize(prod_desc)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

10186


Creating Training set data using feature extraction and bag of words model
==========================================================================

In [20]:
train_data = pd.DataFrame(columns=['Prod_Id','noun','modifier','long_desc'])
train_data['Prod_Id'] = range(1000, 1000 + len(master_txt_corpus))
train_data['noun'] = master_txt_corpus['Noun']
train_data['modifier'] = master_txt_corpus['Modifier']
train_data['long_desc'] = master_txt_corpus['clean_data']
train_data = train_data[train_data['noun']!='NA']
train_data.head()

,Prod_Id,noun,modifier,long_desc
0,1000,GASKET,CONTROL VALVE,VALVES SPARES FOR FLOWSERVE CONTROL VALVE SIZE...
1,1001,OIL,INDUSTRIAL,OIL HYDRUALIC RANDO HD 46 INFLAMMABLE PACKING ...
2,1002,GEAR,HELICAL,UNIT HELICAL GEAR SK1SI75VH IEC 80 MOTORIZED R...
3,1003,SPLICE,CONDUCTOR,SLEEVE CABLE TYPE SB8 DE 3BEXZSB08000 FOR MILL...
4,1004,VALVE,BUTTERFLY,VALVE BUTTERFLY PNEUMATIC OPERATED 8IN WITHOUT...


In [21]:
def cleaning_input_text(text):
    tokens = nltk.word_tokenize(text)
    all_words_bag_temp_1 = nltk.pos_tag(tokens)
    all_words=[]
    for j in range(len(all_words_bag_temp_1)):
        if(all_words_bag_temp_1[j][1] not in ['JJ','MD','PRP','CD','POS','IN'] and all_words_bag_temp_1[j][0].isalpha()):
            all_words.append(all_words_bag_temp_1[j][0])
    clean_text=' '.join(all_words)
    clean_text_1 = clean_text
    del clean_text
    return clean_text_1

In [22]:
'''
train_data.reset_index(drop=True, inplace=True)
df_tf_train_data = pd.DataFrame(columns=['prod_desc','noun','modifier'])

def alphanumeric_removal(text):
    new_text = re.sub(r'\w*\d\w*', '', text)
    return(' '.join(new_text.split()))

df_tf_train_data['prod_desc'] = train_data['long_desc'].apply(alphanumeric_removal)
df_tf_train_data['noun']=train_data['noun']
df_tf_train_data['modifier']=train_data['modifier']
df_tf_train_data.reset_index(drop=True, inplace=True)
df_tf_train_data.head()

'''

train_data.reset_index(drop=True, inplace=True)
df_tf_train_data = pd.DataFrame(columns=['prod_desc','noun','modifier'])
counter = train_data.shape[0]
for i in range(counter):
    inp_clean_data = cleaning_input_text(train_data['long_desc'][i])
    df_tf_train_data = df_tf_train_data.append({'prod_desc': inp_clean_data,'noun':train_data['noun'][i] ,'modifier':train_data['modifier'][i]},ignore_index=True)
df_tf_train_data.reset_index(drop=True, inplace=True)
df_tf_train_data.head()

,prod_desc,noun,modifier
0,VALVES SPARES FOR FLOWSERVE CONTROL VALVE SIZE...,GASKET,CONTROL VALVE
1,OIL HYDRUALIC RANDO HD INFLAMMABLE PACKING STA...,OIL,INDUSTRIAL
2,UNIT HELICAL GEAR IEC MOTORIZED ROLLERS FOR BI...,GEAR,HELICAL
3,SLEEVE CABLE TYPE DE FOR MILL SPOUT FEEDER EQU...,SPLICE,CONDUCTOR
4,VALVE BUTTERFLY PNEUMATIC OPERATED WITHOUT ACT...,VALVE,BUTTERFLY


In [23]:
df_tf_train_data = df_tf_train_data[df_tf_train_data['modifier'] != 'NO MODIFIER']
df_tf_train_data = df_tf_train_data[df_tf_train_data['modifier'] != 'NA']
print(df_tf_train_data.shape)


(21077, 3)


Supervised classifier for noun
================================

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_noun = TfidfVectorizer(max_features=5000, sublinear_tf=True, norm='l2', encoding='latin', ngram_range=(1, 2), stop_words='english')
#tfidf = TfidfVectorizer(max_features = 500, max_df = 0.8, min_df = 0.2, use_idf = True, tokenizer = simp_tokenizer, analyzer = 'word', ngram_range = (1,1))
features_noun = tfidf_noun.fit_transform(df_tf_train_data.prod_desc).toarray()
labels_noun = df_tf_train_data.noun
features_noun.shape

(21077, 5000)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

X_train_noun, X_test_noun, y_train_noun, y_test_noun = train_test_split(df_tf_train_data['prod_desc'], df_tf_train_data['noun'],test_size=0.30, random_state = 0)
count_vect_noun = CountVectorizer()
X_train_counts_noun = count_vect_noun.fit_transform(X_train_noun)
tfidf_transformer_noun = TfidfTransformer()
X_train_tfidf_noun = tfidf_transformer_noun.fit_transform(X_train_counts_noun)
clf_noun = LinearSVC()
clf_noun = clf_noun.fit(X_train_tfidf_noun, y_train_noun)

In [26]:
print(clf_noun.predict(count_vect_noun.transform(["FLANGE FOR CONCENTRIC EDGE ORIFICE TAG NO:40355 MODEL RAISED SPECIFICATION : B16.2 FACE TYPE ORIFICE FLANGE-500 NOMINAL PIPE SIZE:500mm(20""):COMBINE WITH ORIFICE PLATE MODEL:FOR MODEL FOP FLANGE TAP MOUNTING:WELDING NECK, FLANGE MATERIAL: CARBON STEEL-150, FLANGE RATING:ANSI CLASS 150-1993,GASKET MATERIAL:NONASBESTOS, BOLT MATERIAL: SNB7G4, NUT MATERIAL:ASTM A193 Gr4/NN, TAP CONNECTION: 1/2 NPT FEMALE, MAKE: NEWFLOW DATA SHEET ATTACHED"])))
#print(clf_noun.predict(count_vect_noun.transform(["GASKET SPIRAL WOUND SIZE GRAPHITE FILLER MATERIAL  INNER RING MATERIAL MONEL PIPE SIZE 10"" PRESSURE RATING 1500 LB SIZE 10-3/8"" ID X 15"" OD STYLE FLEXITALLIC CG, FURON RW, LAMONS WR THICKNESS 1/16"" WINDING MATERIAL HASTELLOY C NOTE THE MAXIMUM QTY PER  CARDBOARD SHOULD NOT EXCEED FOUR GASKETS ASBESTOS FILLER IS NOT ACCEPTABLE GASKETS FOR FLANGE  size '24 SHALL CONFORM TO ASME B '1647 SERIES A 'MSS-SP-44 FLANGES UNLESS SPECIFIED OTHERWISE  gasket face must clearly show size flange grating manufacturer sansi class '600 TYPE CG WITH  CARBON STEEL OUTER BRING WITH STEEL WINDING METAL WINDING TO SANSI TYPE '316 TI ICB FOR Ll  MARKING AND dimension TO HAPI '601 NOTCHES TO BE HAPPY '1/8 IN DEEP AND LOCATED AT ANY CONVENIENT  LOCATION ALONG THE EDGE OF THE OUTER BRING NOM PIPE SIZE '1-1/4 IN"])))

['GASKET']


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_modifiers = TfidfVectorizer(max_features=50000 , sublinear_tf=True, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features_modifiers = features_noun
labels_modifiers = df_tf_train_data.modifier
features_modifiers.shape

(21077, 5000)

In [28]:
X_train_modifiers, X_test_modifiers, y_train_modifiers, y_test_modifiers = train_test_split(df_tf_train_data['prod_desc'], df_tf_train_data['modifier'],test_size=0.30,random_state = 0)
count_vect_modifiers = CountVectorizer()
X_train_counts_modifiers = count_vect_modifiers.fit_transform(X_train_modifiers)
tfidf_transformer_modifiers = TfidfTransformer()
X_train_tfidf_modifiers = tfidf_transformer_modifiers.fit_transform(X_train_counts_modifiers)
clf_modifiers = LinearSVC().fit(X_train_tfidf_modifiers,y_train_modifiers)

In [29]:
#print(clf_modifiers.predict(count_vect_modifiers.transform(["GASKET spiral wound SHELL ID : 22-5/8 IN OD : 23-3/8 IN 1/8 IN COPPER JACKETED GRAPHITE FILLED TYPE : 1 ITEM C ON DRG. NO :  3142-10" ])))
print(clf_modifiers.predict(count_vect_noun.transform(["FLANGE FOR CONCENTRIC EDGE ORIFICE TAG NO:40355 MODEL RAISED SPECIFICATION : B16.2 FACE TYPE ORIFICE FLANGE-500 NOMINAL PIPE SIZE:500mm(20""):COMBINE WITH ORIFICE PLATE MODEL:FOR MODEL FOP FLANGE TAP MOUNTING:WELDING NECK, FLANGE MATERIAL: CARBON STEEL-150, FLANGE RATING:ANSI CLASS 150-1993,GASKET MATERIAL:NONASBESTOS, BOLT MATERIAL: SNB7G4, NUT MATERIAL:ASTM A193 Gr4/NN, TAP CONNECTION: 1/2 NPT FEMALE, MAKE: NEWFLOW DATA SHEET ATTACHED"])))


['RELIEF VALVE']


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
RandomForestClassifier(n_estimators=10, max_depth=3, random_state=0),
LinearSVC(),
MultinomialNB(),
LogisticRegression(random_state=0)
]
CV = 2
cv_df_nouns = pd.DataFrame(index=range(CV * len(models)))
cv_df_modifiers = pd.DataFrame(index=range(CV * len(models)))

entries_noun = []
entries_modifiers = []

#Cross validation for noun classifiers

for model_noun in models:
    model_name = model_noun.__class__.__name__
    accuracies = cross_val_score(model_noun, features_noun, labels_noun, scoring='accuracy', cv=CV)
    for fold_idx_noun, accuracy_noun in enumerate(accuracies):
        entries_noun.append((model_name, fold_idx_noun, accuracy_noun))
cv_df_noun = pd.DataFrame(entries_noun, columns=['Noun_model_name', 'fold_idx', 'accuracy'])

#Cross validation for modifiers classifiers

for model_modifiers in models:
    model_name = model_modifiers.__class__.__name__
    accuracies = cross_val_score(model_modifiers, features_modifiers, labels_modifiers, scoring='accuracy', cv=CV)
    for fold_idx_modifiers, accuracy_modifiers in enumerate(accuracies):
        entries_modifiers.append((model_name, fold_idx_modifiers, accuracy_modifiers))
cv_df_modifiers = pd.DataFrame(entries_modifiers, columns=['Modifier_model_name', 'fold_idx', 'accuracy'])

print('Evaluating Noun classifiers accuracies: \n',cv_df_noun.groupby('Noun_model_name').accuracy.mean())
print('\n\n Evaluating Noun classifiers accuracies: \n',cv_df_modifiers.groupby('Modifier_model_name').accuracy.mean())


Evaluating Noun classifiers accuracies: 
 Noun_model_name
LinearSVC                 0.936258
LogisticRegression        0.859138
MultinomialNB             0.653611
RandomForestClassifier    0.232706
Name: accuracy, dtype: float64


 Evaluating Noun classifiers accuracies: 
 Modifier_model_name
LinearSVC                 0.889598
LogisticRegression        0.778047
MultinomialNB             0.583174
RandomForestClassifier    0.206919
Name: accuracy, dtype: float64


In [31]:
cv_df_modifiers.groupby('Modifier_model_name').accuracy.mean()

Modifier_model_name
LinearSVC                 0.889598
LogisticRegression        0.778047
MultinomialNB             0.583174
RandomForestClassifier    0.206919
Name: accuracy, dtype: float64

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

model_noun = LinearSVC()
X_train_noun, X_test_noun, y_train_noun, y_test_noun, indices_train_noun, indices_test_noun = train_test_split(features_noun, labels_noun, df_tf_train_data.index, test_size=0.20, random_state=0)
model_noun.fit(X_train_noun, y_train_noun)
y_pred_noun = model_noun.predict(X_test_noun)
print('Accuracy ---->', accuracy_score(y_test_noun, y_pred_noun)*100,'\n')
print('Confusion Matrix ---->\n',confusion_matrix(y_test_noun, y_pred_noun),'\n')
print('Classification Report ----> \n\n',classification_report(y_test_noun, y_pred_noun))

print('\n\n\n')

model_modifier = LinearSVC()
X_train_modifiers, X_test_modifiers, y_train_modifiers, y_test_modifiers, indices_train_modifiers, indices_test_modifiers = train_test_split(features_modifiers, labels_modifiers, df_tf_train_data.index, test_size=0.20, random_state=0)
model_modifier.fit(X_train_modifiers, y_train_modifiers)
y_pred_modifiers = model_modifier.predict(X_test_modifiers)
print('Accuracy ---->', accuracy_score(y_test_modifiers, y_pred_modifiers)*100,'\n')
print('Confusion Matrix ---->\n',confusion_matrix(y_test_modifiers, y_pred_modifiers),'\n')
print('Classification Report ----> \n\n',classification_report(y_test_modifiers, y_pred_modifiers))


Accuracy ----> 94.9003795066 

Confusion Matrix ---->
 [[12  0  0 ...,  0  0  0]
 [ 0  8  0 ...,  0  0  0]
 [ 0  0 19 ...,  0  0  0]
 ..., 
 [ 0  0  0 ..., 57  0  0]
 [ 1  0  0 ...,  0 52  0]
 [ 0  0  0 ...,  0  2  2]] 

Classification Report ----> 

                 precision    recall  f1-score   support

      ABRASIVE       0.86      1.00      0.92        12
      ABSORBER       1.00      1.00      1.00         8
      ACTUATOR       1.00      1.00      1.00        19
       ADAPTER       0.88      0.88      0.88        34
        ANCHOR       1.00      1.00      1.00         2
         ANGLE       1.00      1.00      1.00         2
           BAG       0.88      1.00      0.93         7
          BALL       1.00      0.50      0.67         2
          BAND       1.00      0.50      0.67         2
           BAR       1.00      1.00      1.00        76
       BATTERY       1.00      1.00      1.00         5
          BEAM       1.00      1.00      1.00         1
       BEARING     

Accuracy ----> 90.9392789374 

Confusion Matrix ---->
 [[95  0  0 ...,  0  0  0]
 [ 0 15  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  2  0  0]
 [ 0  0  0 ...,  0 59  0]
 [ 0  0  0 ...,  0  0  6]] 

Classification Report ----> 

                        precision    recall  f1-score   support

                   AC       0.97      1.00      0.98        95
             AC MOTOR       1.00      0.88      0.94        17
          ACCUMULATOR       0.00      0.00      0.00         1
             ACTUATOR       0.00      0.00      0.00         3
             ADHESIVE       1.00      1.00      1.00         1
           ADJUSTABLE       0.75      1.00      0.86         3
                  AIR       0.74      0.89      0.81        38
      AIR CONDITIONER       1.00      1.00      1.00         2
      AIR CONTROL SYS       1.00      1.00      1.00         1
           AIR COOLED       0.00      0.00      0.00         1
             AIR LINE       0.00      0.00      0.00  

In [33]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5

for prod_desc, noun in sorted(df_tf_train_data.items()):
    features_chi2 = chi2(features_noun, labels_noun == noun)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf_noun.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# :",prod_desc)
    print("  \n Most correlated noun unigrams:\n" ,' '.join(unigrams[-N:]))
    print("  \n Most correlated noun bigrams:\n " ,' '.join(bigrams[-N:]))
    
for prod_desc, modifier in sorted(df_tf_train_data.items()):
    features_chi2 = chi2(features_modifiers, labels_modifiers == modifier)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf_noun.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# :",prod_desc)
    print("  \n Most correlated modifiers unigrams:\n" ,' '.join(unigrams[-N:]))
    print("  \n Most correlated modifiers bigrams:\n " ,' '.join(bigrams[-N:]))

# : modifier
  
 Most correlated noun unigrams:
 gate gaskets gasket fluid zm
  
 Most correlated noun bigrams:
  gasket flanged gasket flange gasket face gasket euroseal gasket head
# : noun
  
 Most correlated noun unigrams:
 gate gaskets gasket fluid zm
  
 Most correlated noun bigrams:
  gasket flanged gasket flange gasket face gasket euroseal gasket head
# : prod_desc
  
 Most correlated noun unigrams:
 gate gaskets gasket fluid zm
  
 Most correlated noun bigrams:
  gasket flanged gasket flange gasket face gasket euroseal gasket head
# : modifier
  
 Most correlated modifiers unigrams:
 gate gaskets gasket fluid zm
  
 Most correlated modifiers bigrams:
  gasket flanged gasket flange gasket face gasket euroseal gasket head
# : noun
  
 Most correlated modifiers unigrams:
 gate gaskets gasket fluid zm
  
 Most correlated modifiers bigrams:
  gasket flanged gasket flange gasket face gasket euroseal gasket head
# : prod_desc
  
 Most correlated modifiers unigrams:
 gate gaskets gask

In [34]:
import os
import pickle
f_noun = open('classifier_model_noun.pkl','wb')
pickle.dump(model_noun,f_noun,protocol = 4)
f_noun.close()

f_modifier = open('classifier_model_modifier.pkl','wb')
pickle.dump(model_modifier,f_modifier,protocol = 4)
f_modifier.close()

Testing module
================

In [35]:
end = time.time()
temp = end-start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
print(hours,':',minutes,':',seconds)

0.0 : 6.0 : 53.61425161361694


In [36]:
def noun_identifier(test_sample):
    inp = cleaning_input_text(test_sample)
    dict_noun_modifier={}
    noun_value = ''
    modifier_value=''
    noun = clf_noun.predict(count_vect_noun.transform([inp]))
    noun_value=''.join(noun)
    modifier = clf_modifiers.predict(count_vect_noun.transform([inp]))
    modifier_value=''.join(modifier)
    dict_noun_modifier = {'noun':noun_value,'modifier':modifier_value}
    return dict_noun_modifier

In [1]:
def testing_module(choice,test_input):
    if (choice == '1'):
        df_raw_test = pd.read_csv("Test_Data.csv",encoding = "latin")
        df_raw_test.reset_index(drop=True, inplace=True)
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        counter1=df_raw_test.shape[0]
        
        for i in range(counter1):
            if(i%1000 == 0):
                print(i,'Inside 1 if loop')
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identifier(df_raw_test['Long_Desc'][i])
            noun_value=noun_modifier_dict['noun']
            modifier_value = noun_modifier_dict['modifier']
            concat = noun_value + " | " + modifier_value
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':df_raw_test['Long_Desc'][i],'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)
            
        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        counter = df_raw_test.shape[0]
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        #characteristic_list = set(data_dictionary_mapping.iloc[:,2])
        for i in range(counter):
            if(i%1000 == 0):
                print(i,'Inside 2 if loop')
            noun = output_df['Noun'][i]
            modifier = output_df['Modifier'][i]
            char_list = data_dictionary_mapping[(data_dictionary_mapping['Noun'] == noun) & (data_dictionary_mapping['Modifier'] == modifier) ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            counter1 = len(char_values_all)
            for j in range(counter1):
                char_value = char_values_all[j]
                char_flag=None
                word_list=char_value.split()
                search = word_list[0]
                prod_desc=output_df['Material_Desc'][i]
                clean_string = prod_desc.replace(':', ' ').replace('-', ' ')
                list_of_words = clean_string.split()
                if (search in list_of_words):
                    last_index =list_of_words.index(search) + len(word_list)
                    if (len(list_of_words) > last_index):
                        char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                        output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                if (char_flag is None):
                    output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
            output_char_value_df = output_char_value_df.fillna(method='ffill')
            output_char_value_df = output_char_value_df.tail(1)

            if (output_char_value_df.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = np.NaN
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            output_char_value_df=output_char_value_df.iloc[0:0]
        
        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)
        new = pd.DataFrame()
        new = final_output_df.iloc[:,5:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        
        for i in range(new.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 3 if loop')       
            short_desc=''
            long_desc=''
            val=''
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc =  str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        
        
        df_formatter=pd.concat([df_temp1,df],axis=1)
        
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 4 if loop')

            new_long_desc = "*** " + str(df_formatter.loc[i,'Noun']) + ' , ' + str(df_formatter.loc[i,'Modifier']) + " *** " + '\n\n' + str(df_formatter.loc[i,'long_description'])
            modi_search = df_formatter.loc[i,'Modifier']
            if(modi_search in list(modifier_dict['Modifier'])):
                short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                new_short_desc =  str(df_formatter.loc[i,'Noun']) + ' , ' + str(short_modifier) + ': ' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            else:
                new_short_desc = str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            df_formatter.loc[i,'long_description'] = new_long_desc
            df_formatter.loc[i,'short_description'] = new_short_desc
        
        df_exception = pd.DataFrame()
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 5 if loop')

            mat_desc=df_temp1.loc[i,'Material_Desc'].replace(',',' ')
            #mat_desc= mat_desc.split()
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(df_formatter.loc[i,'long_description'])]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)
        
            
        output_dataframe=pd.DataFrame()
        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_exception,new_output_df],axis=1)
        output_dataframe.to_csv('final_output_df_file_import.csv')
        #display(output_dataframe)
        print('Please check you local file system for output generated : File ----> final_output_df_file_import.csv')
        return output_dataframe

    if (choice == '2'):
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        for i in range(1):
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identifier(test_input)
            noun_value = noun_modifier_dict['noun']
            modifier_value = noun_modifier_dict['modifier']
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':test_input,'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)

        def ngram_model(text,n_val,char_value):
            from nltk import ngrams
            sentence = text
            n = n_val
            ngrams = ngrams(sentence.split(), n)
            for grams in ngrams:
                str1 = " ".join(list(grams))
                return_val='NA'
                if (str1.upper() == char_value.upper()):
                    return_val=char_value.upper()
                    return (return_val)
                    break
                else:
                    return_val='NA'

        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        for i in range(1):
            characteristic_list = set(data_dictionary_mapping.iloc[:,2])
            noun = output_df['Noun'][i]
            modifier = output_df['Modifier'][i]
            char_list= data_dictionary_mapping[data_dictionary_mapping['Noun'] == noun ]
            char_list= char_list[char_list['Modifier'] == modifier ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            word_tokens = [w for w in nltk.word_tokenize(output_df['Material_Desc'][i])]
            counter1 = len(char_values_all)
            for j in range(counter1):
                char_value = char_values_all[j]
                values = char_list[char_list['Characteristic'] == char_value]['TheValue']
                for k in values:
                    n_gram = len(k.split())
                    char_flag = ngram_model(output_df['Material_Desc'][i],n_gram,k)
                    if (char_flag is not None):
                        output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                        break
                if (char_flag is None):
                    word_list=char_value.split()
                    search = word_list[0]
                    prod_desc=output_df['Material_Desc'][i]
                    clean_string1 = prod_desc.replace(':', ' ')
                    clean_string = clean_string1.replace('-', ' ')
                    list_of_words = clean_string.split()
                    if (search in list_of_words):
                        last_index =list_of_words.index(search) + len(word_list)
                        if (len(list_of_words) > last_index):
                            char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                            output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                if (char_flag is not None):
                    output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                else:
                    output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
            output_char_value_df = output_char_value_df.fillna(method='ffill')
            output_char_value_df = output_char_value_df.tail(1)
            if (output_char_value_df_dummy.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = 'NaN'
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            output_char_value_df=output_char_value_df.iloc[0:0]

        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)    
        new = pd.DataFrame()
        new = final_output_df.iloc[:,4:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        for i in range(new.shape[0]):
            short_desc=''
            long_desc=''
            gskt = final_output_df.loc[i,'Noun']
            modi = final_output_df.loc[i,'Modifier']
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc = str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        df_formatter=pd.concat([df_temp1,df],axis=1)
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)

        for i in range(df_formatter.shape[0]):
            new_long_desc = "*** " + str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + " ***" + '\n\n' + str(df_formatter.loc[i,'long_description'])
            modi_search = df_formatter.loc[i,'Modifier']
            if(modi_search in list(modifier_dict['Modifier'])):
                short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                new_short_desc =  str(df_formatter.loc[i,'Noun']) + ',' + str(short_modifier) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
                print(new_short_desc,'\n\n')
            else:
                new_short_desc = str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
                print(new_short_desc,'\n\n')
            df_formatter.loc[i,'long_description'] = new_long_desc
            df_formatter.loc[i,'short_description'] = new_short_desc

        df_exception = pd.DataFrame()
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 5 if loop')

            mat_desc=df_temp1.loc[i,'Material_Desc'].replace(',',' ')
            #mat_desc= mat_desc.split()
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(df_formatter.loc[i,'long_description'])]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)

        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_exception,new_output_df],axis=1)
        output_dataframe.to_csv('final_output_df_text_inp.csv')
        print('Please check you local file system for output generated : File ----> final_output_df_text_inp.csv')
        return output_dataframe

In [2]:
print("Select operation to input a test sample")
print("1.File import")
print("2.Manual Text input")

Select operation to input a test sample
1.File import
2.Manual Text input


In [3]:
import time
start = time.time()

choice = input("Enter choice(1/2):")
if choice == '1':
    print('Make sure you have feeded your input in the Test_Data.csv')
    test_input=''
    pred_output_df = pd.DataFrame()
    pred_output_df = testing_module(choice,test_input.upper())
if choice == '2':
    #test_input = input('Enter your test sample\n')
    test_input = "SPIRAL WOUND GASKET SIZE: 1""X 900/1500 LBS;WITH EXTERNAL GUIDE RING (STYLE-CG), WINDING MATERIAL:316L FILLER MATERIAL GRAPHITE AS PER ASME B 16.20 TO FIT FOR ANSI B 16.5 FLANGES"
    pred_output_df = testing_module(choice,test_input.upper())
    #display(pred_output_df)
end = time.time()
temp = end-start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
print(hours,':',minutes,':',seconds)

Enter choice(1/2):2


NameError: name 'pd' is not defined

In [ ]:
pred_output_df = pred_output_df.sort_values(['Noun','Modifier'], ascending=[True,True])
#pred_output_df.drop(['NA'],axis=1,inplace=True)
n_noun_modifier= pd.DataFrame()
n_noun_modifier = pred_output_df.loc[:,'Noun':'Modifier'].drop_duplicates()
n_noun_modifier.reset_index(drop=True, inplace=True)
writer = pd.ExcelWriter('final_output_df_file_import_excel.xlsx', engine='xlsxwriter')
#writer = pd.ExcelWriter('pandas_conditional.xlsx', engine='xlsxwriter')

for i in range(n_noun_modifier.shape[0]):
    noun=n_noun_modifier.loc[i,'Noun'].replace('/','_')
    modifier=n_noun_modifier.loc[i,'Modifier'].replace('/','_')
    df_name = str(noun) + "_" + str(modifier)
    name = n_noun_modifier.loc[i,'Noun'].replace(' ','_').replace('/','_') + "_" + n_noun_modifier.loc[i,'Modifier'].replace(' ','_').replace('/','_')
    df_taxonomy = pd.DataFrame(index=None)
    df_taxonomy = pred_output_df[pred_output_df['Noun'] == noun]
    df_taxonomy = df_taxonomy[df_taxonomy['Modifier'] == modifier]
    df_taxonomy.dropna(axis='columns',inplace=True,how='all')
    df_taxonomy = df_taxonomy.replace('NA','')
    df_taxonomy.reset_index(drop=True, inplace=True)
    df_taxonomy.columns = map(str.upper, df_taxonomy.columns)
    workbook  = writer.book
    df_taxonomy.to_excel(writer,sheet_name=name)
    worksheet = writer.sheets[name]
    start_row = 1
    start_col = 8
    end_row = df_taxonomy.shape[0]
    end_col = df_taxonomy.shape[1]
    format0 = workbook.add_format({'bg_color': '#FFA07A'})
    format1 = workbook.add_format({'bg_color': '#90EE90'})
    format2 = workbook.add_format({'font_color': '#FFFFFF','bg_color': '#0000FF'})
    cell_format = workbook.add_format()
    cell_format.set_align('left')
    cell_format.set_align('top')
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'blanks', 'format': format0})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'no_blanks', 'format': format1})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'format': cell_format})
    worksheet.conditional_format(0, 0, 0, end_col, {'type':'no_blanks', 'format': format2})
writer.save()
